# FILE ORGANIZER

___

## 1. Organizzazione dei file

In questo notebook si documenterà la creazione di uno script per organizzare i file contenuti nella cartella "files".
Durante l'esecuzione lo script mostrerà log tipo:
```
bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:doc size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B
```

I file verranno organizzati in sottocartelle della cartella "files" con una struttura tipo:
```
- files
    - audio
        - song1.mp3
        - song2.mp3
    - docs
        - ciao.txt
        - pippo.odt
    - images
        - bw.png
        - daffodil.jpg
        - eclipse.png
        - trump.jpeg
    - recap.csv
```

Il file recap.csv contine i log creati dallo script.

___

In [ ]:
# Librerie necessarie

In [ ]:
# Controllo: Nella cartella file sono presenti le sottocartelle? Se non sono presenti vengono create.

In [ ]:
# Per ogni file stampare a schermo nome, tipo e dimensioni. Si sposta il file nella cartella corretta. Infine si salva il log dell'operazione nel file recap.csv.

## 2. Conversione dello script da notebook a eseguibile

## 3. Utilizzo di NumPy e PIL per mostrare informazionni sulle immagini